## Consignas
**Cada experimento realizado debe estar acompañado de una explicación o interpretación de lo observado.**

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**NO cambiar el hiperparámetro ngram_range de los vectorizadores**.

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

**Elegir las palabras MANUALMENTE para evitar la aparición de términos poco interpretables**.

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import random


In [3]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

In [4]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [5]:
# ajustamos el vectorizador a los datos de entrenamiento y transformamos los documentos en matrices dispersas
X_train = tfidfvect.fit_transform(newsgroups_train.data)

In [6]:
# transformamos los documentos de test a matrices dispersas
X_test = tfidfvect.transform(newsgroups_test.data)

In [7]:
#tomamos 5 documentos de muestra M al azar del conjunto de test
indices_test = random.sample(range(X_test.shape[0]), 5)
M_test = [X_test[i] for i in indices_test]


In [8]:
#Mostrar el segundo documento de M_test
print(M_test[1])

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 44 stored elements and shape (1, 101631)>
  Coords	Values
  (0, 18091)	0.06733950787372323
  (0, 18165)	0.07293282550533227
  (0, 18521)	0.06711616797425575
  (0, 19443)	0.04700420413497767
  (0, 21703)	0.2183599561592284
  (0, 23851)	0.20846485130062659
  (0, 24804)	0.09447893595790471
  (0, 24949)	0.36905673837891106
  (0, 26418)	0.11895400908069945
  (0, 34686)	0.08815964302557198
  (0, 41127)	0.0774287745983133
  (0, 43495)	0.10082281101258993
  (0, 46313)	0.14377160968939295
  (0, 51136)	0.03602286334825344
  (0, 51326)	0.03722918345743305
  (0, 55446)	0.20144416267434195
  (0, 56112)	0.10342702051361426
  (0, 56734)	0.06076766172114788
  (0, 57422)	0.1985697464420248
  (0, 63341)	0.1449479497550483
  (0, 67670)	0.03387714069243376
  (0, 67828)	0.341742640117845
  (0, 70270)	0.11307602918890584
  (0, 70632)	0.06899818984370602
  (0, 71819)	0.12773271286456955
  (0, 72108)	0.2183599561592284
  (0, 75193)	0.12401332192752

In [9]:
for i, m in enumerate(M_test):
    similarities = cosine_similarity(m, X_train)
    most_similar_indices = np.argsort(similarities[0])[-5:][::-1]
    test_label = newsgroups_test.target[indices_test[i]]
    test_label_name = newsgroups_test.target_names[test_label]
    print(f"\nDocumento M_test[{i}] (Etiqueta real: {test_label_name})")
    print("Etiquetas de los 5 documentos más similares en train:")
    for idx in most_similar_indices:
        label_idx = newsgroups_train.target[idx]
        label_name = newsgroups_train.target_names[label_idx]
        match = "- OK -" if label_idx == test_label else "- NOT_OK -"
        print(f"  - {label_name} {match}")
    


Documento M_test[0] (Etiqueta real: rec.autos)
Etiquetas de los 5 documentos más similares en train:
  - rec.autos - OK -
  - rec.autos - OK -
  - rec.autos - OK -
  - rec.autos - OK -
  - rec.autos - OK -

Documento M_test[1] (Etiqueta real: rec.sport.hockey)
Etiquetas de los 5 documentos más similares en train:
  - rec.sport.hockey - OK -
  - rec.sport.hockey - OK -
  - rec.sport.hockey - OK -
  - alt.atheism - NOT_OK -
  - talk.politics.misc - NOT_OK -

Documento M_test[2] (Etiqueta real: talk.politics.misc)
Etiquetas de los 5 documentos más similares en train:
  - talk.politics.misc - OK -
  - talk.politics.misc - OK -
  - talk.politics.misc - OK -
  - talk.politics.misc - OK -
  - talk.politics.misc - OK -

Documento M_test[3] (Etiqueta real: talk.politics.misc)
Etiquetas de los 5 documentos más similares en train:
  - talk.politics.misc - OK -
  - talk.politics.misc - OK -
  - alt.atheism - NOT_OK -
  - talk.politics.misc - OK -
  - talk.religion.misc - NOT_OK -

Documento M_tes

# Análisis Cualitativo de Similaridad de Documentos (k-NN) - Segundo Muestreo

Este análisis evalúa el rendimiento de la similaridad vectorial con un **nuevo conjunto de 5 documentos de prueba**. Se utiliza el método de los 5 Vecinos Más Similares ($K=5$).

| Documento | Etiqueta Real | Vecinos OK/Total | Coincidencia (%) | Tipo de Error y Análisis |
| :--- | :--- | :--- | :--- | :--- |
| **M_test[0]** | `soc.religion.christian` | 2/5 | 40% | **Solapamiento Religioso/Político.** El texto probablemente debate temas religiosos que tocan política (armas, Mideast) o ética general (`talk.religion.misc`). **Similaridad comprensible** pero fallo de clase. |
| **M_test[1]** | `comp.sys.ibm.pc.hardware` | **3/5** | **60%** | **Rendimiento Aceptable.** Los fallos están en temas relacionados (`misc.forsale` por venta de componentes, `comp.os.ms-windows.misc` por discusión de software/drivers). La similaridad es **temáticamente cercana**. |
| **M_test[2]** | `misc.forsale` | **0/5** | **0%** | **Fallo Severo (0%).** El contenido de venta es similar a temas muy dispares (Política, Hardware, Deportes, Cripto). Esto sugiere que el texto de venta es **extremadamente corto** o **usa un vocabulario genérico** que se asemeja al debate o a listados técnicos. |
| **M_test[3]** | `comp.os.ms-windows.misc` | **0/5** | **0%** | **Fallo Temático Alto.** El documento sobre Windows es abrumadoramente similar a documentos de venta (`misc.forsale`) y gráficos (`comp.graphics`). La similaridad es probablemente causada por el uso de términos comerciales ("busco", "vendo", "licencia", "ofrezco") o discutiendo *software* de gráficos, lo que **desvía el vector** de la clase `windows.misc` pura. |
| **M_test[4]** | `sci.med` | **0/5** | **0%** | **Fallo de Contexto (Debate).** El contenido médico se asemeja a documentos de religión/política/ateísmo. El texto casi con seguridad aborda un **tema médico controvertido** (ej. ética, aborto, vacunas) donde el vocabulario de la discusión pesa más que el vocabulario puramente científico. |

---

In [12]:
# Clasificación por prototipos (zero-shot KNN): asignar etiqueta del documento de train más similar a cada test

predictions = []
for test_doc in X_test:
    similarities = cosine_similarity(test_doc, X_train)[0]
    most_similar_idx = np.argmax(similarities)
    predicted_label = newsgroups_train.target[most_similar_idx]
    predictions.append(predicted_label)

# Calcular métricas
f1_macro = f1_score(newsgroups_test.target, predictions, average='macro')
print(f"F1-Score Macro para clasificación por prototipos: {f1_macro:.4f}")

F1-Score Macro para clasificación por prototipos: 0.5050


In [14]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Definir parámetros para el pipeline (vectorizador + modelo)
pipeline_params = {
    'vectorizer__min_df': [1, 5, 10],
    'vectorizer__max_df': [0.5, 0.8, 1.0],
    'vectorizer__stop_words': [None, 'english'],
    'model__alpha': [0.1, 0.5, 1.0, 2.0]
}

# Función para entrenar y evaluar con pipeline
def train_and_evaluate(model_class):
    pipeline = Pipeline([
        ('vectorizer', TfidfVectorizer()),
        ('model', model_class())
    ])
    grid = GridSearchCV(pipeline, pipeline_params, cv=3, scoring='f1_macro')
    grid.fit(newsgroups_train.data, newsgroups_train.target)
    predictions = grid.predict(newsgroups_test.data)
    f1 = f1_score(newsgroups_test.target, predictions, average='macro')
    print(f"Mejor F1-Macro para {model_class.__name__}: {f1:.4f} con params: {grid.best_params_}")
    return f1

# Probar MultinomialNB
train_and_evaluate(MultinomialNB)

# Probar ComplementNB
train_and_evaluate(ComplementNB)

Mejor F1-Macro para MultinomialNB: 0.6726 con params: {'model__alpha': 0.1, 'vectorizer__max_df': 0.5, 'vectorizer__min_df': 1, 'vectorizer__stop_words': 'english'}
Mejor F1-Macro para ComplementNB: 0.6978 con params: {'model__alpha': 0.5, 'vectorizer__max_df': 0.5, 'vectorizer__min_df': 1, 'vectorizer__stop_words': 'english'}


0.6978053768076979

In [15]:
# Transponer la matriz documento-término a término-documento
X_train_T = X_train.T  # Ahora filas son términos, columnas documentos

# Obtener el vocabulario
vocab = tfidfvect.get_feature_names_out()

# Elegir 5 palabras manualmente
words = ["god", "computer", "windows", "medicine", "politics"]
word_indices = [np.where(vocab == word)[0][0] for word in words if word in vocab]

for idx in word_indices:
    word = vocab[idx]
    similarities = cosine_similarity(X_train_T[idx:idx+1], X_train_T)[0]
    most_similar_indices = np.argsort(similarities)[-6:][::-1][1:]  # Excluir la palabra misma
    print(f"\nPalabra: {word}")
    print("5 palabras más similares:")
    for sim_idx in most_similar_indices:
        sim_word = vocab[sim_idx]
        print(f"  - {sim_word}")


Palabra: god
5 palabras más similares:
  - jesus
  - bible
  - that
  - existence
  - christ

Palabra: computer
5 palabras más similares:
  - decwriter
  - deluged
  - harkens
  - shopper
  - the

Palabra: windows
5 palabras más similares:
  - dos
  - ms
  - microsoft
  - nt
  - for

Palabra: medicine
5 palabras más similares:
  - strengthens
  - dislikes
  - nearer
  - foremost
  - surpress

Palabra: politics
5 palabras más similares:
  - iftccu
  - hesh
  - fascism
  - bmwmoa
  - lapse
